# Lab 3 — recommender systems

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
plt.style.use("ggplot")

What does the data look like?

In [ ]:
!hdfs dfs -cat /ix/ml-20m/ratings.txt | tail -n 2

In [ ]:
data = sc.textFile("/ix/ml-20m/ratings.txt").map(json.loads)

## Exercise 3.4 Basic statistics :

Users :

In [ ]:
users_ratings = data.map(lambda x: (x['userId'], x['rating']))
users_counts = users_ratings.reduceByKey(add).collect()

Movies

In [ ]:
movies_ratings = data.map(lambda x: (x['movieId'], x['rating']))
movies_counts = movies_ratings.reduceByKey(add).collect()

Now let's plot :

In [ ]:
cumulative_users = np.cumsum(users_counts)
cumulative_movies = np.cumsum(movies_counts)

fig, (axe1, axe2) = plt.subplots(ncols=2)

axe1.plot(cumulative_users)
axe1.set_title("Users")
axe1.set_xlabel('User')
axe1.set_ylabel('Ratings')

axe2.plot(cumulative_movies)
axe2.set_title("Movies")
axe2.set_xlabel('User')
axe2.set_ylabel('Ratings')

axe2.loglog()
axe1.loglog()

plt.show()

Some users have a lot of ratings as well as movies and users with little number of ratings. The distributions of the ratings isn't balanced uniformly.

## Exercise 3.5 Partitioning the dataset :

In [ ]:
personnal_ratings = sc.parallelize("something ?") ## add the personnal ranking somehow

newData = data.union(personnal_ratings)

training = newData.filter(lambda x: (x['timestamp'] % 10) > 1)
test = newData.filter(lambda x: (x['timestamp'] % 10) <= 1)

### Baseline recommender system

#### Baseline model

In [ ]:
mu = training.map(lambda x: x[2]).mean()

In [ ]:
## find how to get all_user_bias and all_item_bias

In [ ]:
def user_bias(userId):
    if userId in all_user_bias.keys():
        return all_user_bias[userId]
    
    return ## Use mean of biases

def item_bias(movieId):
    if movieId in all_item_bias.keys():
        return all_item_bias[movieId]
    
    return ## Use mean of biases

In [ ]:
def predict(user, movie):
    return ((user, movie), mu + user_bias(user) + item_bias(movie))

#### Evaluation

In [ ]:
def error(rdd):
    ranks_and_preds = test.map(lambda x: ((x[0], x[1]), x[2])).join(rdd)
    err = ## Implement the error calculation
    
    return err

### Matrix-factorization model

In [ ]:
from pyspark.mllib.recommendation import ALS

Lower rank implies simpler model, thus a faster training. Higher rank increases the amount of parameters used in the learning process, thus the probability of overfitting increases with the rank.

In [ ]:
## Choose the rank ? (Shall we use 10 ?)

#### Regularization

In [ ]:
param = 10e-4
model = ALS.train(training, rank, lambda_=param)
prediction = model.predictAll("SOME DATA HERE").map(lambda x: ((x[0], x[1]), x[2]))


error = error(prediction)
print('Error with lambda=10^-4: ', error)

param2 = 10.0
model2 = ALS.train(training, rank, lambda_=param2)
prediction2 = model.predictAll("SOME DATA HERE").map(lambda x: ((x[0], x[1]), x[2]))


error2 = error(prediction)
print('Error with lambda=10.0: ', error2)

In [ ]:
## Find optimal lambda below
optimal_lambda = VALUE

#### Recommendation

In [ ]:
model = ALS.train("DATA", rank=?, lambda_=optimal_lambda)

movies = sc.textFile("/ix/ml-20m/movies.txt").map(json.loads)
ids_to_title = movies.map(itemgetter("movieId", "title")).collectAsMap()

In [ ]:
## Do the recommendation for both users

#### Visualisation

In [ ]:
new_model = ALS.train(training, rank=2, lambda_=optimal_lambda)
selected_movies = sorted(pickle.load(open('selected-movies.pickle', 'rb'), encoding='utf-8'))

In [ ]:
• Create an interactive plot that embeds the movies along the 2 directions defined by the
factorization.
• Describe what you observe. Can you give a name to the dimensions? Do you recognize
cluster of movies that are alike?